In [8]:
!pip -q install -U ultralytics roboflow python-dotenv pyyaml

from ultralytics import YOLO
from dotenv import load_dotenv
import os, yaml

# Local runs directory (beside your notebook)
RUN_ROOT = "./runs"
os.makedirs(RUN_ROOT, exist_ok=True)
print("Saving all outputs under:", os.path.abspath(RUN_ROOT))


Saving all outputs under: /Users/Marcello/code/Arina-W/roadsafe-ai-detect-locate-notify/notebooks/Marx/damage_detection/runs


In [9]:
# Downloading dataset from roboflow
# .env should contain: ROBOFLOW_API_KEY=xxxxxxxxxxxx
load_dotenv()
api_key = os.getenv("ROBOFLOW_API_KEY", "").strip()
assert api_key, "ROBOFLOW_API_KEY missing. Add it to .env next to the notebook."

from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project  = rf.workspace("khumbu").project("road-damage-rwhhe")
version  = project.version(1)
dataset  = version.download("yolov11")   # -> e.g. ./road-damage-1

DATA_YAML = os.path.join(dataset.location, "data.yaml")
print("DATA_YAML:", DATA_YAML)


loading Roboflow workspace...
loading Roboflow project...
DATA_YAML: /Users/Marcello/code/Arina-W/roadsafe-ai-detect-locate-notify/notebooks/Marx/damage_detection/road-damage-1/data.yaml


In [10]:
# we reuse this best.pt file so we don't need to retrain the model
model = YOLO("best.pt")

In [11]:
# See class mapping
print("Class mapping:")
for k, v in model.names.items():
    print(f"{k} → {v}")

Class mapping:
0 → 0
1 → 1
2 → 2
3 → 3
4 → 4


In [12]:
val_metrics = model.val(
    data=DATA_YAML,
    split="val",
    plots=False,      # <- disable plots so lzma isn't needed
    project=RUN_ROOT,
    name="roadsafe_v11s_val",
    exist_ok=True
)
test_metrics = model.val(
    data=DATA_YAML,
    split="test",
    plots=False,      # <- same here
    project=RUN_ROOT,
    name="roadsafe_v11s_test",
    exist_ok=True
)

print("Val saved to:",  val_metrics.save_dir)
print("Test saved to:", test_metrics.save_dir)

Ultralytics 8.3.187 🚀 Python-3.12.9 torch-2.2.2 CPU (Intel Core(TM) i9-9980HK 2.40GHz)
YOLO11s summary (fused): 100 layers, 9,414,735 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.2 ms, read: 164.5±43.0 MB/s, size: 89.2 KB)
val: Scanning /Users/Marcello/code/Arina-W/roadsafe-ai-detect-locate-notify/notebooks/Marx/damage_detection/road-damage-1/valid/labels.cache... 260 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 260/260 301248.4it/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 0.22it/s 1:16s
                   all        260        584      0.777      0.687       0.74      0.466
                     0        115        170      0.747      0.765      0.776      0.483
                     1         87        143      0.762       0.47      0.588      0.279
                     2         55         78      0.905      0.858      0.899      0.671
                     3         77 

In [13]:
import glob

# Get first 20 validation images
val_images = sorted(glob.glob(os.path.join(os.path.dirname(DATA_YAML), "valid", "images", "*")))[:20]

pred_val = model.predict(
    source=val_images,
    conf=0.25,
    save=True,
    project=RUN_ROOT,
    name="roadsafe_v11s_pred_valid",
    exist_ok=True
)
print("Val predictions →", pred_val[0].save_dir)

# Get first 20 test images
test_images = sorted(glob.glob(os.path.join(os.path.dirname(DATA_YAML), "test", "images", "*")))[:20]

pred_test = model.predict(
    source=test_images,
    conf=0.25,
    save=True,
    project=RUN_ROOT,
    name="roadsafe_v11s_pred_test",
    exist_ok=True
)
print("Test predictions →", pred_test[0].save_dir)



0: 800x800 1 2, 1 3, 305.5ms
1: 800x800 1 3, 305.5ms
2: 800x800 3 0s, 1 4, 305.5ms
3: 800x800 1 2, 305.5ms
4: 800x800 1 0, 305.5ms
5: 800x800 2 2s, 305.5ms
6: 800x800 1 1, 305.5ms
7: 800x800 3 0s, 305.5ms
8: 800x800 1 2, 1 4, 305.5ms
9: 800x800 1 2, 305.5ms
10: 800x800 1 2, 1 4, 305.5ms
11: 800x800 2 2s, 305.5ms
12: 800x800 1 2, 2 4s, 305.5ms
13: 800x800 1 4, 305.5ms
14: 800x800 1 3, 305.5ms
15: 800x800 2 3s, 305.5ms
16: 800x800 1 0, 305.5ms
17: 800x800 2 3s, 1 4, 305.5ms
18: 800x800 1 2, 305.5ms
19: 800x800 2 3s, 305.5ms
Speed: 5.2ms preprocess, 305.5ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/roadsafe_v11s_pred_valid
Val predictions → runs/roadsafe_v11s_pred_valid

0: 800x800 1 2, 1 3, 1 4, 288.0ms
1: 800x800 1 3, 288.0ms
2: 800x800 3 0s, 288.0ms
3: 800x800 2 0s, 288.0ms
4: 800x800 1 2, 288.0ms
5: 800x800 1 2, 288.0ms
6: 800x800 1 2, 288.0ms
7: 800x800 1 2, 288.0ms
8: 800x800 1 3, 288.0ms
9: 800x800 3 3s, 288.0ms
10: 800x800 3 3s, 288.0m

In [14]:
with open(DATA_YAML) as f:
    print(f.read())


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 5
names: ['0', '1', '2', '3', '4']

roboflow:
  workspace: khumbu
  project: road-damage-rwhhe
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/khumbu/road-damage-rwhhe/dataset/1
